Alignment_and_Deployment_Lab_v2.ipynb

# Lab: Alignment y Deployment de Modelos Generativos
## De Raw Model a Production-Ready con SFT

### Objetivos del Lab:
1. Entender la progresión de un modelo Base a uno SFT (Supervised Fine-Tuned).
2. Experimentar con técnicas de fine-tuning eficiente (LoRA).
3. Comparar un modelo fine-tuned con recursos limitados contra un modelo oficial.

---
## Setup Inicial


In [ ]:
# Instalación de dependencias
!pip install -q transformers datasets peft accelerate bitsandbytes trl torch

In [ ]:
import torch
import gc
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    TrainingArguments,
    BitsAndBytesConfig
)
from datasets import load_dataset
from peft import LoraConfig, get_peft_model, TaskType, PeftModel
from trl import SFTTrainer
import pandas as pd
import time
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Verificar hardware
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"VRAM: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

## Utilidades Globales

In [ ]:
def clear_memory():
    """Limpiar memoria GPU y RAM"""
    gc.collect()
    torch.cuda.empty_cache()
    if torch.cuda.is_available():
        torch.cuda.synchronize()
    print("Memoria limpiada")

In [ ]:
def get_gpu_memory():
    """Obtener uso actual de memoria GPU"""
    if torch.cuda.is_available():
        return torch.cuda.memory_allocated() / 1e9
    return 0

In [ ]:
def generate_text(model, tokenizer, prompt, max_length=200, temperature=0.7):
    """
    Generar texto usando el modelo
    """
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            inputs.input_ids,
            max_length=max_length,
            temperature=temperature,
            do_sample=True,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id,
            attention_mask=inputs.attention_mask
        )

    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return generated_text[len(prompt):].strip()

# PARTE 1: El Problema - Los Modelos Base

### 1.1 EJERCICIO 1: Define tus Preguntas de Evaluación

**Instrucciones:**
1. Crea una lista de 10 preguntas de evaluación en la variable `eval_questions`.
2. Las preguntas deben ser variadas para probar diferentes capacidades (conocimiento, redacción, etc.).

In [ ]:
# TODO (EJERCICIO 1): Define tu propia lista de 10 preguntas
eval_questions = [
    "¿Qué es machine learning?",
    # "Pregunta 2...",
    # "Pregunta 3...",
    # ...
]

In [ ]:
print("Preguntas de evaluación definidas:")
for i, q in enumerate(eval_questions, 1):
    print(f"{i}. {q}")

### 1.2 Cargar y Evaluar Modelo Base

In [ ]:
model_name_base = "Qwen/Qwen2.5-1.5B"

In [ ]:
print(f"Cargando modelo base: {model_name_base}")
tokenizer = AutoTokenizer.from_pretrained(model_name_base, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [ ]:
model_base = AutoModelForCausalLM.from_pretrained(
    model_name_base,
    torch_dtype=torch.float16,
    device_map="auto",
    trust_remote_code=True
)
print(f"Modelo cargado. Memoria GPU usada: {get_gpu_memory():.2f} GB")

In [ ]:
def evaluate_model_manual(model, tokenizer, questions, model_name="Model"):
    print(f"\n{'='*60}\nEVALUACIÓN: {model_name}\n{'='*60}\n")
    results = []
    for i, question in enumerate(questions, 1):
        print(f"[{i}/{len(questions)}] Pregunta: {question}")
        response = generate_text(model, tokenizer, question, max_length=150)
        print(f"Respuesta: {response[:200]}{'...' if len(response) > 200 else ''}")
        print("-" * 60)
        results.append({'question': question, 'response': response, 'model': model_name})
    return pd.DataFrame(results)

In [ ]:
# Guardar resultados y limpiar memoria
results_base = evaluate_model_manual(model_base, tokenizer, eval_questions, "BASE")
del model_base
clear_memory()

### 1.3 Análisis del Modelo Base

**Instrucciones:**
Basado en las respuestas guardadas en `results_base`, completa la siguiente tabla para tu análisis.

In [ ]:
#Ejemplo
evaluation_ejemplo = {
    'question': eval_questions,
    'understood': [False, False, False, True, False, False, False, False, False, False],
    'coherence': [2, 2, 1, 3, 1, 2, 2, 1, 2, 2],
    'helpfulness': [1, 1, 1, 2, 1, 1, 1, 1, 1, 1],
    'observations': [
        'Continúa con texto técnico sin responder', 'No da explicación clara, divaga',
        'No estructura en lista, texto confuso', 'Responde correctamente por casualidad',
        'No sigue formato de email', 'Texto técnico incomprensible',
        'No da receta estructurada', 'Definición vaga y confusa',
        'Mezcla conceptos sin claridad', 'No resume, genera texto irrelevante'
    ]
}

In [ ]:
#Completar
evaluation_base = {
    'question': eval_questions,
    'understood': [None] * len(eval_questions),
    'coherence': [None] * len(eval_questions),
    'helpfulness': [None] * len(eval_questions),
    'observations': [''] * len(eval_questions)
}
df_eval_base = pd.DataFrame(evaluation_base)
print("\nTABLA DE EVALUACIÓN - MODELO BASE (Completa los valores None)")
print(df_eval_base.head())

## PARTE 2: Supervised Fine-Tuning (SFT)

### 2.1 Preparar Dataset y Configuración LoRA

In [ ]:
print("Cargando dataset Alpaca...")
dataset = load_dataset("tatsu-lab/alpaca", split="train")
train_dataset = dataset.select(range(1000))
print(f"Usando {len(train_dataset)} ejemplos para SFT")

In [ ]:
def format_alpaca_instruction(sample):
    if sample['input'] and sample['input'].strip():
        prompt = f"""### Instruction:\n{sample['instruction']}\n\n### Input:\n{sample['input']}\n\n### Response:\n{sample['output']}"""
    else:
        prompt = f"""### Instruction:\n{sample['instruction']}\n\n### Response:\n{sample['output']}"""
    return {"text": prompt}

In [ ]:
formatted_dataset = train_dataset.map(format_alpaca_instruction)
print("Dataset formateado.")

In [ ]:
lora_config = LoraConfig(
    r=1, # Probar otros valores de 1 a 32 (si tienen tiempo mas),
    lora_alpha=16,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_dropout=0.05, bias="none", task_type=TaskType.CAUSAL_LM,
)
print("Configuración LoRA creada.")

### 2.2 EJERCICIO 2: Experimentar con LoRA Rank

**Instrucciones:**
Modifica el valor de 'r' en la configuración LoRA y observa cómo cambia el número de parámetros entrenables. Descomenta y ejecuta el siguiente bloque.


In [ ]:
# TODO (EJERCICIO 2): Prueba diferentes valores de r
def count_trainable_params(r_value):
    lora_config_test = LoraConfig(
        r=r_value,
        lora_alpha=r_value * 2,
        target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
        task_type=TaskType.CAUSAL_LM)

    model_temp = AutoModelForCausalLM.from_pretrained(
        model_name_base, torch_dtype=torch.float16, device_map="auto", trust_remote_code=True)

    model_temp = get_peft_model(model_temp, lora_config_test)
    trainable = sum(p.numel() for p in model_temp.parameters() if p.requires_grad)
    total = sum(p.numel() for p in model_temp.parameters())

    del model_temp
    clear_memory()
    return trainable, total

In [ ]:
r_values = [4, 8, 16, 32] # probar agregar mas valores de r
for r in r_values:
    trainable, total = count_trainable_params(r)
    print(f"r={r}: {trainable:,} params entrenables ({100*trainable/total:.2f}%)")

### 2.3 Entrenar y Evaluar con SFT

In [ ]:
print("Cargando modelo para SFT...")
model_sft = AutoModelForCausalLM.from_pretrained(model_name_base, torch_dtype=torch.float16, device_map="auto", trust_remote_code=True)
model_sft = get_peft_model(model_sft, lora_config)

In [ ]:
training_args = TrainingArguments(
    output_dir="./qwen-sft-lora", per_device_train_batch_size=4, gradient_accumulation_steps=4,
    num_train_epochs=1, learning_rate=2e-4, fp16=True, logging_steps=25, save_strategy="epoch",
    optim="paged_adamw_8bit", lr_scheduler_type="cosine", warmup_steps=50, report_to="none", remove_unused_columns=False,
)

In [ ]:
trainer = SFTTrainer(
    model=model_sft,
    train_dataset=formatted_dataset,
    args=training_args,
    formatting_func=lambda x: x["text"],
)

In [ ]:
print("Iniciando entrenamiento SFT...")
start_time = time.time()
trainer.train()
end_time = time.time()
print(f"Entrenamiento completado en {(end_time - start_time)/60:.1f} minutos")

In [ ]:
model_sft.save_pretrained("./qwen-sft-adapters")
tokenizer.save_pretrained("./qwen-sft-adapters")
print("Adapters LoRA guardados en ./qwen-sft-adapters")

In [ ]:
# Guardar resultados y limpiar memoria
results_sft = evaluate_model_manual(model_sft, tokenizer, eval_questions, "SFT")
del model_sft, trainer
clear_memory()

## PARTE 3: Comparación Final

Ahora, comparemos nuestro modelo SFT con el modelo oficial de Qwen.

### 3.1 Cargar y Evaluar Modelo Instruct Oficial

In [ ]:
print("Cargando Qwen2.5-1.5B-Instruct (modelo oficial)...")
model_instruct = AutoModelForCausalLM.from_pretrained(
    "Qwen/Qwen2.5-1.5B-Instruct",
    torch_dtype=torch.float16,
    device_map="auto",
    trust_remote_code=True
)
print("Modelo oficial cargado.")

In [ ]:
# Guardar resultados y limpiar memoria
results_instruct = evaluate_model_manual(model_instruct, tokenizer, eval_questions, "INSTRUCT")
del model_instruct
clear_memory()

### 3.2 EJERCICIO 3: Comparación Triple (Base vs SFT vs Official)

**Instrucciones:**
1. Ejecuta la celda para ver las respuestas de los tres modelos lado a lado.
2. Compara la calidad, detalle y estilo de tu modelo SFT con el modelo oficial.
3. Reflexiona sobre qué tan cerca llegaste del modelo oficial con recursos limitados.

In [ ]:
print("\n" + "="*100)
print("COMPARACIÓN FINAL: BASE vs NUESTRO SFT vs OFFICIAL INSTRUCT")
print("="*100)
print("\nRecuerda: Entrenamos con solo 1000 ejemplos SFT.")
print("El modelo oficial fue entrenado con millones de ejemplos.\n")

In [ ]:
for i, question in enumerate(eval_questions):
    print(f"\n{'='*100}")
    print(f"Pregunta: {question}")
    print(f"{'='*100}")

    print("\nMODELO BASE:")
    print(results_base.iloc[i]['response'])

    print("\nNUESTRO SFT:")
    print(results_sft.iloc[i]['response'])

    print("\nOFFICIAL INSTRUCT:")
    print(results_instruct.iloc[i]['response'])

    print("-" * 100)

### 3.3 Reflexión Final

**Preguntas para discutir:**
1. ¿Qué tan cerca llegamos del modelo oficial con recursos limitados?
2. ¿En qué aspectos nuestro modelo SFT es competitivo?
3. ¿Dónde se nota más la diferencia en la escala de entrenamiento?
4. ¿Qué mejorarías si tuvieras más recursos (datos, compute, tiempo)?

---
## Resumen del Lab

### Lo que aprendimos:
1. **SFT enseña el formato de "instruction-following" de forma eficiente con LoRA.**
2. **La escala de datos importa, pero la técnica es accesible.** Con pocos datos se logran grandes mejoras.

### Próximos Pasos:
- Experimentar con más datos de SFT.
- Probar con un dataset más específico a un dominio.
- Evaluar con métricas automáticas (ROUGE, BLEU).

---
## Limpieza Final

In [ ]:
clear_memory()
print("Lab completado!")
print("Archivos generados:")
print("  - ./qwen-sft-adapters/: Adapters LoRA después de SFT")
print("¡Excelente trabajo!")